<a href="https://colab.research.google.com/github/mayanashakeerkhan/Diabetes-Prediction/blob/main/HCL_GUVI_HACKTHON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber python-docx langdetect spacy
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 119.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import gradio as gr

def show_image(image):
    # Simply return the image to display it
    return image

with gr.Blocks() as demo:
    gr.Markdown("## 📷 Real-Time Image Upload Preview")

    image_input = gr.Image(type="pil", label="Upload Image")
    image_output = gr.Image(label="Uploaded Image Preview")

    image_input.change(
        fn=show_image,
        inputs=image_input,
        outputs=image_output
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://add495ba3031ceada4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import re
import spacy
from langdetect import detect

# Load NLP model
nlp = spacy.load("en_core_web_sm")
def detect_language(text):
    try:
        return "Hindi" if detect(text) == "hi" else "English"
    except:
        return "English"
def extract_clauses(text):
    clauses = re.split(r"\n\d+\.|\n[A-Z][A-Za-z ]{3,}:", text)
    return [c.strip() for c in clauses if len(c.strip()) > 40]
def extract_entities(text):
    doc = nlp(text)
    entities = {
        "Parties": [],
        "Dates": [],
        "Money": [],
        "Jurisdiction": []
    }

    for ent in doc.ents:
        if ent.label_ == "ORG":
            entities["Parties"].append(ent.text)
        elif ent.label_ == "DATE":
            entities["Dates"].append(ent.text)
        elif ent.label_ == "MONEY":
            entities["Money"].append(ent.text)
        elif ent.label_ == "GPE":
            entities["Jurisdiction"].append(ent.text)

    return entities
def classify_clause_type(clause):
    c = clause.lower()
    if "shall" in c or "must" in c:
        return "Obligation"
    if "may" in c:
        return "Right"
    if "shall not" in c or "must not" in c:
        return "Prohibition"
    return "General"
HIGH_RISK = [
    "indemnify", "penalty", "non-compete",
    "terminate without notice", "exclusive",
    "unilateral", "ip transfer"
]

MEDIUM_RISK = [
    "arbitration", "jurisdiction",
    "auto-renewal", "lock-in"
]

def assess_risk(clause):
    c = clause.lower()
    for word in HIGH_RISK:
        if word in c:
            return "High"
    for word in MEDIUM_RISK:
        if word in c:
            return "Medium"
    return "Low"
def explain_clause_plain(risk):
    if risk == "High":
        return "This clause strongly favors the other party and may expose your business to legal or financial risk."
    elif risk == "Medium":
        return "This clause may cause issues depending on how it is enforced."
    else:
        return "This clause is standard and generally safe."
def suggest_alternative(risk):
    if risk == "High":
        return "Negotiate mutual obligations, reduce penalties, or add clear limits."
    elif risk == "Medium":
        return "Clarify timelines, renewal terms, and jurisdiction."
    else:
        return "No changes required."

def analyze_contract(contract_text):
    language = detect_language(contract_text)
    clauses = extract_clauses(contract_text)
    entities = extract_entities(contract_text)

    clause_results = []
    risk_count = {"Low": 0, "Medium": 0, "High": 0}

    for i, clause in enumerate(clauses):
        risk = assess_risk(clause)
        risk_count[risk] += 1

        clause_results.append({
            "Clause No": i + 1,
            "Type": classify_clause_type(clause),
            "Risk": risk,
            "Explanation": explain_clause_plain(risk),
            "Suggestion": suggest_alternative(risk),
            "Text": clause[:250]
        })

    overall_risk = "Low"
    if risk_count["High"] > 0:
        overall_risk = "High"
    elif risk_count["Medium"] > 2:
        overall_risk = "Medium"

    return {
        "Detected Language": language,
        "Overall Contract Risk": overall_risk,
        "Extracted Entities": entities,
        "Clause Analysis": clause_results
    }


In [ ]:
contract = """
The Vendor shall indemnify the Company against all losses.
The Employee shall not engage in any competing business for two years.
Either party may terminate this agreement with 30 days notice.
This agreement shall be governed by the laws of India.
"""

result = analyze_contract(contract)

print("Language:", result["Detected Language"])
print("Overall Risk:", result["Overall Contract Risk"])
print("Entities:", result["Extracted Entities"])

for c in result["Clause Analysis"]:
    print("\nClause", c["Clause No"])
    print("Type:", c["Type"])
    print("Risk:", c["Risk"])
    print("Explanation:", c["Explanation"])
    print("Suggestion:", c["Suggestion"])


Language: English
Overall Risk: High
Entities: {'Parties': ['Company'], 'Dates': ['two years', '30 days'], 'Money': [], 'Jurisdiction': ['India']}

Clause 1
Type: Obligation
Risk: High
Explanation: This clause strongly favors the other party and may expose your business to legal or financial risk.
Suggestion: Negotiate mutual obligations, reduce penalties, or add clear limits.
